# **Exploratory Data Analysis (EDA) of PHEE Data**

Four aims during the EDA of the PHEE Data.

1. Description and relationship of medical/data terms. Quantify data quality
2. Demographic Distribution of the patients
3. Distribution of text data tokens, parts of speech, prevalent terms

In [1]:
import pickle

# **Load Data**

In [13]:
def pkl_load_dict(filename):
  """
    Load the dictionary data from a pickle file into a variable

    @P:
    filename (str): Name of the pkl file
    varname : Name of the df to save the pkl data

    @R:
    varname : Containing the pkl data
  """

  with open(filename, 'rb') as handle:
      df = pickle.load(handle)

  return df

df=pkl_load_dict('/content/drive/MyDrive/PHEE/output/data_df.pkl')
df.head(20)

,index,Text,Subject,Treatment,Potential_therapeutic_event,Drug,Effect,Adverse_event,Race,Age,Gender,Population,Disorder,Duration,Time_elapsed,Route,Freq,Dosage,Combination.Drug
0,12891225_2,Aprepitant is a neurokinin(1) receptor antagon...,,Aprepitant added to conventional therapy with ...,enhances,Aprepitant||corticosteroid||5-hydroxytryptamin...,prevention of chemotherapy-induced nausea and ...,,,,,,,,,,,,
1,9007910_1,Dihydropyrimidine dehydrogenase deficiency: a ...,,5-fluorouracil-based chemotherapy,,5-fluorouracil,severe adverse reactions,to,,,,,,,,chemotherapy,,,
2,8467620_3,We report one case of non-Hodgkin lymphoma in ...,"a patient, with a 30-year history of rheumatoi...",low dose methotrexate weekly over a 10-month p...,,methotrexate,non-Hodgkin lymphoma,taking,,,,a,,10-month period.,,,weekly,low dose,
3,8589490_1,Supravenous hyperpigmentation in association w...,,CHOP chemotherapy,,CHOP,Supravenous hyperpigmentation,association,,,,,,,,chemotherapy,,,
4,10646879_3,We conclude that MB is an effective treatment ...,,ifosfamide||MB,treatment,ifosfamide||MB,encephalopathy,induced,,,,,,,,,,,
5,11510798_2,"Diarrhoea, T-CD4+ lymphopenia and bilateral pa...",male 60 yrs of age,oxaliplatinum and 5-fluorouracil,,oxaliplatinum||5-fluorouracil,"Diarrhoea, T-CD4+ lymphopenia and bilateral pa...",developed,,60 yrs of age,male,,,,,,,,
6,12504711_1,A paradoxical ocular effect of brimonidine.\n,,brimonidine,,brimonidine,paradoxical ocular effect,of,,,,,,,,,,,
7,16216617_2,While on a maximal dose of phenylephrine she d...,she,maximal dose of phenylephrine,,phenylephrine,positive U waves,developed,,,she,,,,,,,maximal dose,
8,12659609_2,OBJECTIVE: To report a case of severe chloroqu...,high-grade chloroquine-resistant Plasmodium vivax,chloroquine,,chloroquine,severe chloroquine toxicity,toxicity,,,,,,,,,,,
9,868978_2,The ocular motor disturbances are probably an ...,,5-FU,,5-FU,The ocular motor disturbances neurotoxicity,expression,,,,,,,,,,,


# **Description and relationship of medical/data terms. Quantify data quality**

*** *For the frequency, include the relative frequencies - include a table*

Description of select attributes

**Subject** Patients involved in the medical event

The five most common subjects in the dataset are:

**Treatment** The therapy administered to
the patients

The five most common treatments in the dataset are:

**Potential_therapeutic_event** Abbreviated as PTE. Denotes potentially beneficial effects of medical therapies.

The five most common PTE's in the dataset are:

**Drug** A prescribed medication that causes a physiological effect 

The ten most common treatments in the dataset are:

**Effect** The outcome of the treatment

**Adverse_event** Abbreviated as ADE. Denotes potentially harmful effects of medical therapies.

The five most common PTE's in the dataset are:

**Race** Categorization of humans based on shared physical or social qualities into groups

Distribution of races in the dataset

**Age** Number of years that a person has lived

Distribution of ages in the dataset

**Gender** Sex of patients

Distribution of gender in the dataset

**Population**

**Disorder**

**Duration**

**Time_elapsed**

**Route**

**Freq**

**Dosage**

**Combination.Drug**

**Text** Sentences extracted from biomedical literature, MEDLINE case reports, annotated with information
relevant to pharmacovigilance.


> **Ex** Diarrhoea, T-CD4+ lymphopenia and bilateral patchy pulmonary infiltrates developed in a male 60 yrs of age, who was treated with oxaliplatinum and 5-fluorouracil for unresectable rectum carcinoma.



In [11]:
print("An example sentence extracted from a MEDLINE case report: \n {} \n".format(df['Text'].iloc[5]))

An example sentence extracted from a MEDLINE case report: 
 Diarrhoea, T-CD4+ lymphopenia and bilateral patchy pulmonary infiltrates developed in a male 60 yrs of age, who was treated with oxaliplatinum and 5-fluorouracil for unresectable rectum carcinoma.
 



In [12]:
#5 Most common values in dataframe column function

# **Demographic Distribution of the Patients**

# **Distribution of Text Data Tokens, Parts of Speech, Prevalent Terms**